In [1]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import pandas as pd

import random

Inicialmente se solicita simular un espacio con dimensiones 7 unidades de alto por 7 de ancho. En este espacio hay 4 personas contagiadas, quienes esparcen la enfermedad, y 20 personas sanas, ambas tanto enfermos como sanos estan en ubicaciones aleatorias.

Las personas que propagan el contagio pueden moverse en todas direcciones, pero solamente pueden contagiar a personas que se encuentren en celdas adyacentes. Cuando una persona sana es contagiada permanece en su lugar, sin embargo es capaz de identificar que adquirió la enfermedad.

In [40]:
#remover tuplas vacias
def Remove(tuples):
    tuples = [t for t in tuples if t]
    return tuples

#delete tuples where first element is equal or minor to input
def deleteLeft(tuples, value):
    tuples = [t for t in tuples if t[1] >= value]
    return tuples

def deleteRight(tuples, value):
    tuples = [t for t in tuples if t[1] <= value]
    return tuples

def deleteUp(tuples, value):
    tuples = [t for t in tuples if t[0] >= value]
    return tuples

def deleteDown(tuples, value):
    tuples = [t for t in tuples if t[0] <= value]
    return tuples

In [43]:
class Persona(ap.Agent):


    def setup(self):
            self.grid = self.model.grid
            self.random = self.model.random
            self.moving = 0 # 0: none, 1: moving, 2: stopped
            self.state = 0 #0 person, 1 virus, 2: contaminado
            self.currentPosition = ()
            self.nextMove = ""
    
    
    
    def nearbyCars(self):
        neighbours = self.grid.neighbors(self, distance=1)
        neighbours = neighbours.to_list()
        
        neighbours = Remove(neighbours.currentPosition) #regresa los vecinos de la posicion actual con distancia de 6

        neighbours = sorted(neighbours, key=lambda tup: tup[0])

        return neighbours
    
    def possibleRight(self):
        #check if neighbor is not at the right of the car
        traffic = self.nearbyCars()

        traffic = deleteLeft(traffic, self.currentPosition[1]) #regresa los vecinos unicamente de la derecha

        traffic = deleteDown(traffic, self.currentPosition[0]) #regresa los vecinos unicamente de abajo
        
        traffic = deleteUp(traffic, self.currentPosition[0]) #regresa los vecinos unicamente de arriba

        #print(self.currentPosition,end = ' ') 
        #print(traffic)

        if traffic:
            for car in traffic:
                #check if the nearest car to the right is 4 spaces away
                if car[1] > self.currentPosition[1] + 1:
                #if car[1] > self.currentPosition[1] + 4 :
                    return True
                else:
                    return False
        else:
            return True
    
    def possibleLeft(self):
        #check if neighbor is not at the left of the car
        traffic = self.nearbyCars()

        traffic = deleteRight(traffic, self.currentPosition[1])

        traffic = deleteDown(traffic, self.currentPosition[0]) #regresa los vecinos unicamente de abajo
        
        traffic = deleteUp(traffic, self.currentPosition[0]) #regresa los vecinos unicamente de arriba

        #print(self.currentPosition,end = ' ') 
        #print(traffic)

        if traffic:
            for car in traffic:
                #check if the nearest car to the right is 4 spaces away
                if car[1] < self.currentPosition[1] - 1:
                    return True
                else:
                    return False
        else:
            return True

    def possibleUp(self):
        #check if neighbor is not at the right of the car
        traffic = self.nearbyCars()

        traffic = deleteLeft(traffic, self.currentPosition[1]) #regresa los vecinos unicamente de la derecha
        traffic = deleteRight(traffic, self.currentPosition[1])
        traffic = deleteDown(traffic, self.currentPosition[0]) #regresa los vecinos unicamente de abajo

        if traffic:
            for car in traffic:
                #check if the nearest car to the right is 4 spaces away
                if car[0] < self.currentPosition[0] - 1:
                    return True
                else:
                    return False
        else:
            return True

    def possibleDown(self):
        #check if neighbor is not at the right of the car
        traffic = self.nearbyCars()

        traffic = deleteLeft(traffic, self.currentPosition[1]) #regresa los vecinos unicamente de la derecha
        traffic = deleteRight(traffic, self.currentPosition[1])
        traffic = deleteUp(traffic, self.currentPosition[0]) #regresa los vecinos unicamente de abajo

        if traffic:
            for car in traffic:
                #check if the nearest car to the right is 4 spaces away
                if car[0] > self.currentPosition[0] - 1:
                    return True
                else:
                    return False
        else:
            return True


    def moveRight(self):
        #FALTA check if there is a neighbor to the right
        self.orientation = 1

        if(self.possibleRight()):
            self.grid.move_to(self,(self.currentPosition[0],self.currentPosition[1]+1))
            self.currentPosition = self.grid.positions[self]

    def moveLeft(self):
        #FALTA check if there is a neighbor to the left
        self.orientation = 3
        if(self.possibleLeft()):
            self.grid.move_to(self,(self.currentPosition[0],self.currentPosition[1]-1))
            self.currentPosition = self.grid.positions[self]

    def moveUp(self):
        #FALTA check if there is a neighbor to the left
        self.orientation = 0
        if(self.possibleUp()):
            self.grid.move_to(self,(self.currentPosition[0]-1,self.currentPosition[1]))
            self.currentPosition = self.grid.positions[self]

    def moveDown(self):
        self.orientation = 2
        #FALTA check if there is a neighbor to the left
        if(self.possibleDown()):
            self.grid.move_to(self,(self.currentPosition[0]+1,self.currentPosition[1]))
            self.currentPosition = self.grid.positions[self]

    def updateState(self):
        self.state = 1

    def contaminated(self): #checka si esta junto a un contamnado
        neighbours = self.grid.neighbors(self, distance=1)
        neighbours = neighbours.to_list()

        for neighbour in neighbours:
            if neighbour.state == 1:
                return False
            else:
                return True

        return True



    def move(self):
        if not self.currentPosition:
            self.currentPosition = self.grid.positions[self]
        
        movement = self.random.randint(0, 3) #movimiento en el grid

        if(self.state != 2): #si no esta infectado ni contaminado     #0 person, 1 virus, 2: contaminado
            if(self.contaminated() == False  and self.state == 0):
                self.state = 2
            else:
                if movement == 0:
                    self.moveRight()
                elif movement == 1:
                    self.moveLeft()
                elif movement == 2:
                    self.moveUp()
                elif movement == 3:
                    self.moveDown()


In [32]:
class VrusModel(ap.Model):
    def setup(self):
        s = self.p.size #size grid
        n = 4 #number of viruses
        b = 20    #number of people

        total = n+b

        #Create grid and agents
        self.grid = ap.Grid(self, (s, s), track_empty=True)

        #Create agents
        self.agents = ap.AgentList(self, total, Persona)

        self.grid.add_agents(self.agents, random=True, empty=True)

        for i in range(n):
            self.agents[i].state = 1

    def update(self):
        self.agents.move()    
    
    def step(self):
        # Move agents
        self.agents.move()  
        

In [33]:
parameters = {
    'size': 7, # Size of the grid
    'steps': 50  # Maximum number of steps
}

In [44]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    group_grid = model.grid.attr_grid('state')
    color_dict = {0:'#7FC97F' ,1:'#ffff', 2:"#fe4450", None:'#343131'} #verde personas, blanco virus, rojo contaminado
    ap.gridplot(group_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title("Modelo Virus")

fig, ax = plt.subplots()
model = VrusModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())

# Diseño del ambiente

- Dimensiones 7x7
- agente inicia en posiciones aleatorias
- Los agentes se pueden mover en todas direcciones
- Los agentes contaminados no se pueden mover
- Los agentes no se pueden mover en celdas ocupadas

El tiempo será manejado por steps, cada step representará un fotograma, se espera que la animación en unity sea a 30 cuadros por segundo por lo que cada segundo requerirá de 30 steps. Esto es relevante ya que a partir de esto se calculará la velocidad que deberán seguir con el fin de que una vez enviado a Unity esta se vea correctamente. Agentpy maneja este espacio como malla y la información la almacenará como una matriz. El modelo será de tipo multiagente, contando con cada uno de los agentes (coche, bicicleta y camion) y con varias representaciones de los mismos. 

El codigo funciona al generar una lista de agentes. Estos agentes pueden ser o no virus. Se generan unicamente 4 virus y lo demas son personas. tienen los mismos comportamientos sin embargo si un humano es infectado este pasa a quedarse inmovil. esto se puso con una condicion booleana a partir de las adyacente como vecinos. Se pueden mover pero necesitan verificar las adyacentes para ver si estan libres. Las posiciones son libres y aleatorias

Los estados principales son contaminado, virus o persona
Las metas son la contaminacion total o supervivencia total
Las acciones son movimientos y contaminacion
el modelo de transicion se basa en verificaciones perceptuales. No hay como tal una comunicacion directa sino que es por observacion
